#Import Package<br>
許多與pretrain_nn類似<br>
models: 要訓練哪些模型(會依序訓練)<br>
備註: 此程式會依據frame_index.txt來決定要使用哪些frame數的CCTV來訓練與測試

In [ ]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
from datetime import datetime
from datetime import timedelta
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from keras.utils import Sequence
from keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import random as rn
from models_2 import *
np.random.seed(666)
rn.seed(666)
tf.set_random_seed(666)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

Import model<br>
包括 c3d、c3g25、c4g12、c5g6、c6g3、lrcn

In [ ]:
%run models_2.ipynb

#Class Define<br>
論文第10頁<br>
參考 : 英文 https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly <br>
      中文 https://blog.csdn.net/m0_37477175/article/details/79716312 

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, list_IDs_1, labels, fnum, peak, shuffle=True):
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs_1 = list_IDs_1
        self.fnum = fnum
        self.shuffle = shuffle
        self.on_epoch_end()
        self.peak = peak

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.list_IDs_1) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data"""
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        list_IDs_temp_1, y = [self.list_IDs_1[k] for k in indexes],\
                             [self.labels[k] for k in indexes]
        X1 = self.__data_generation(list_IDs_temp_1, indexes)
        return X1, np.array(y, dtype=int)

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.list_IDs_1))
        if self.shuffle == True:
            np.random.seed(666)
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp_1, y_index):
        """Generates data containing batch_size samples"""
        X1 = []
        for ID, y_ID in zip(list_IDs_temp_1, y_index):
            # Store sample
            x_1 = list(np.load(ID.split(" ")[0] + '.npy')) #different with pretrain_nn , split
            if len(x_1) >= 120:
                x_1 = x_1[70:120]
            else:
                x_1 = x_1[:frame_num]
            while len(x_1) < self.fnum:
                x_1.append(np.zeros_like(x_1[0]))
            x_1 = add_noises(x_1, ID.split(" ")[1])         #different with pretrain_nn , add noise
            X1.append(x_1)  
        X1 = np.array(X1)
        return X1

class FrameInterval:
    """Classify data by frame number"""
    def __init__(self):
        self.train_df = pd.DataFrame(columns=['data'], dtype=str)
        self.test_df = pd.DataFrame(columns=['data'], dtype=str)
        self.val_df = pd.DataFrame(columns=['data'], dtype=str)

    def add_train(self, data, datatime):
        df = pd.DataFrame(columns=['data'], dtype=str)
        df.loc[datatime] = [data]
        self.train_df = self.train_df.append(df)
        self.train_df.index = pd.to_datetime(self.train_df.index)

    def add_test(self, data, datatime):
        self.test_df.loc[datatime] = [data]
        self.test_df.index = pd.to_datetime(self.test_df.index)

    def add_val(self, data, datatime):
        self.val_df.loc[datatime] = [data]
        self.val_df.index = pd.to_datetime(self.val_df.index)

#Define add_noises function<br> 
輸入input格式為 D:/wcs/cctv/nfbCCTV-N5-S-21.048-M/20190729_1644 + " 0" or " 1" <br>
["D:/wcs/cctv/nfbCCTV-N5-S-21.048-M/20190729_1644","0"] or ["D:/wcs/cctv/nfbCCTV-N5-S-21.048-M/20190729_1644","1"] <br>

In [ ]:
def add_noises(images, ind):             #將tag為1的加入noise 的training set 為0的維持一樣
    if ind == '0':  
        """Original"""
        return images
    elif ind == '1':
        """Adding noise"""
        resize = []
        rand1 = np.random.randint(0, 4)  #隨機選擇其中一邊
        rand2 = np.random.randint(0, 16) #裁減0~15個pixel
        for img in images:
            if rand1 == 0:
                img = img[rand2:, :]
                img = np.concatenate((img, np.zeros((rand2, 106, 1))), axis=0) #向上或向下偏移
            elif rand1 == 1:
                img = img[:, rand2:]
                img = np.concatenate((img, np.zeros((72, rand2, 1))), axis=1)  #向左或向右偏移
            elif rand1 == 2:
                img = img[:72-rand2, :]
                img = np.concatenate((np.zeros((rand2, 106, 1)), img), axis=0)
            else:
                img = img[:, :106-rand2]
                img = np.concatenate((np.zeros((72, rand2, 1)), img), axis=1)
            resize.append(img)
        return resize


#define compare_mae

In [ ]:
def compare_mae(train, test, frame_num, dir, model_temp):
    X_train = []
    y_train_index = []
    X_test = []
    y_test_index = []
    X_val = []
    y_val_index = []
    for t in test:
        X_test += FrameList[t].test_df['data'].tolist()
        y_test_index += FrameList[t].test_df.index
        X_val += FrameList[t].val_df['data'].tolist()
        y_val_index += FrameList[t].val_df.index
    for t in train:
        X_train += FrameList[t].train_df['data'].tolist()
        y_train_index += FrameList[t].train_df.index
    y_train = vd_df.loc[y_train_index, feature].values
    y_test = vd_df.loc[y_test_index, feature].values
    y_val = vd_df.loc[y_val_index, feature].values
    print("Train Length:", len(X_train), file=open(total_name, 'a'))
    print("Test Length:", len(X_test), file=open(total_name, 'a'))
    print("Val Length:", len(X_val), file=open(total_name, 'a'))

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    X_val = np.array(X_val)
    y_val = np.array(y_val)

    training_generator = DataGenerator(X_train, y_train, frame_num, dir, True)
    testing_generator = DataGenerator(X_test, y_test, frame_num, dir, False)
    validation_generator = DataGenerator(X_val, y_val, frame_num, dir, True)

    history = model_temp.fit_generator(generator=training_generator, validation_data=validation_generator,
                                       epochs=epochs, steps_per_epoch=len(X_train) // batch_size,
                                       validation_steps=len(X_val) // batch_size,
                                       max_queue_size=200, verbose=1, callbacks=[early_stopping], workers=4)
    print("Epochs: %d" % len(history.history['loss']), file=open(total_name, 'a'))
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(loc=1)
    plt.tight_layout()
    plt.grid()
    plt.savefig(
        "plot/" + name + "/" + cctv_Id + "/" + dir + "/loss.png")
    testPrdct = model_temp.predict_generator(generator=testing_generator, max_queue_size=200, workers=4, verbose=1,
                                             steps=len(X_test) // batch_size)
    testPrdct = np.squeeze(testPrdct)
    mae = abs(y_test[:len(testPrdct)] - testPrdct).mean()
    print("MAE:", "%.2f" % mae, file=open(total_name, 'a'))
    plt.figure(num=None, figsize=(50, 5))
    std = abs(y_test[:len(testPrdct)] - testPrdct).std()
    plt.title('MAE: %.2f, σ: %.2f' % (mae, std), fontsize=30)
    plt.tick_params(labelsize=30)
    plt.plot(y_test, lw=2, label='Target')
    plt.plot(testPrdct, lw=2, label='Predict')
    plt.tight_layout()
    plt.grid()
    # plt.title("MAE: %.3f" % mae_3)
    plt.legend(loc=1, prop={'size': 20})
    plt.savefig(
        "plot/" + name + "/" + cctv_Id + "/" + dir + "/total.png")
    plt.close('all')
    plt.clf()
    plt.cla()
    return model_temp, testPrdct

#Basic setting<br>
day1, train_dayLast: 訓練時間<br>
test_day1, dayLast: 測試時間<br>
val_day1, val_dayLast: 驗證時間<br>
feature: 選擇真值(Spd_Max, Spd_Min, Spd_Avg)

In [ ]:
"""VDs and CCTVs"""
#vds = ['nfbVD-N5-N-16.900-M-PS', 'nfbVD-N5-N-18.313-M-PS', 'nfbVD-N5-N-22.510-M-PS', 'nfbVD-N5-N-25.310-M-PS',
#       'nfbVD-N5-S-18.312-M-PS', 'nfbVD-N5-S-19.677-M-PS', 'nfbVD-N5-S-21.063-M-PS', 'nfbVD-N5-S-23.910-M-PS']
# vds = ['nfbVD-N5-S-18.312-M-PS', 'nfbVD-N5-S-19.677-M-PS', 'nfbVD-N5-S-21.063-M-PS', 'nfbVD-N5-S-23.910-M-PS']
vds = ["nfbVD-N5-S-21.063-M-PS", "nfbVD-N5-S-23.910-M-PS"]

#cctvs = ['nfbCCTV-N5-N-16.915-M', 'nfbCCTV-N5-N-18.308-M', 'nfbCCTV-N5-N-22.514-M', 'nfbCCTV-N5-N-25.309-M',
#         'nfbCCTV-N5-S-18.339-M', 'nfbCCTV-N5-S-19.7-M', 'nfbCCTV-N5-S-21.048-M', 'nfbCCTV-N5-S-23.896-M']
# cctvs = ['nfbCCTV-N5-S-18.339-M', 'nfbCCTV-N5-S-19.7-M', 'nfbCCTV-N5-S-21.048-M', 'nfbCCTV-N5-S-23.896-M']
cctvs = ['nfbCCTV-N5-S-21.048-M', 'nfbCCTV-N5-S-23.896-M']

# Todo: choose data set, tune training params
features = ['Spd_Max', 'Spd_Avg', 'Spd_Min']


day1 = datetime.strptime('20190728 0600', "%Y%m%d %H%M")
day1_str = day1.strftime("%Y%m%d %H%M")
train_dayLast = datetime.strptime('20190729 2359', "%Y%m%d %H%M")

test_day1 = datetime.strptime('20190728 0600', "%Y%m%d %H%M")
test_day1_str = test_day1.strftime("%Y%m%d %H%M")
dayLast = datetime.strptime('20190729 2359', "%Y%m%d %H%M")
dayLast_str = dayLast.strftime("%Y%m%d %H%M")

val_day1 = datetime.strptime('20190729 0600', "%Y%m%d %H%M")
val_dayLast = val_day1 + timedelta(days=6) - timedelta(minutes=1)


# day1 = datetime.strptime('20190322 0600', "%Y%m%d %H%M")
# day1_str = day1.strftime("%Y%m%d %H%M")
# train_dayLast = datetime.strptime('20190414 2359', "%Y%m%d %H%M")

# test_day1 = datetime.strptime('20190517 0600', "%Y%m%d %H%M")
# test_day1_str = test_day1.strftime("%Y%m%d %H%M")
# dayLast = datetime.strptime('20190601 2359', "%Y%m%d %H%M")
# dayLast_str = dayLast.strftime("%Y%m%d %H%M")

# val_day1 = datetime.strptime('20190602 0600', "%Y%m%d %H%M")
# val_dayLast = val_day1 + timedelta(days=6) - timedelta(minutes=1)


#Todo<br>
model: 要訓練的神經網路模型<br>
filename: 選擇vd真值的csv檔<br>
備註: 此程式會將frame數分為兩個model去訓練，並分別向左右延伸，找到最佳的訓練與測試集<br>
frame_num : 取50frame 論文第10頁<br>
frame_index.txt欄位說明: [CCTV,peak1 train, peak1 test, peak2 train, peak2 test] *此檔案須根據pretrain_nn的結果來手動編輯<br>

In [ ]:
feature = 'Spd_Avg'
frame_num = 50
batch_size = 32
epochs = 100
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='auto',
                               verbose=0, restore_best_weights=True)

model_count = 0

"""Models to train"""
#models = [c3d(), c6g3(), c5g6(), c4g12(), c3g25(), lrcn()]
models = [c5g6(), c4g12(), c3g25(), lrcn()]

for name, model in models:
    if not os.path.exists("plot/" + name):
        os.makedirs("plot/" + name)
    total_name = 'plot/' + name + '/log.txt'
    open(total_name, 'w').close()
    for Id, (cctv_Id, vd_Id) in enumerate(zip(cctvs, vds)):
        print(cctv_Id)
        print(cctv_Id, file=open(total_name, 'a'))
        if not os.path.exists("plot/" + name + "/" + cctv_Id + "/peak_1"):
            os.makedirs("plot/" + name + "/" + cctv_Id + "/peak_1")
            os.makedirs("plot/" + name + "/" + cctv_Id + "/peak_2")
        else:
            for f in os.listdir("plot/" + name + "/" + cctv_Id + "/peak_1"):
                os.remove("plot/" + name + "/" + cctv_Id + "/peak_1/" + f)
            for f in os.listdir("plot/" + name + "/" + cctv_Id + "/peak_2"):
                os.remove("plot/" + name + "/" + cctv_Id + "/peak_2/" + f)
        vdDir = 'D:/vd/' + vd_Id + '/vd_speed/'
        cctvDir = 'D:/wcs/cctv/' + cctv_Id + "/"
        filename = vdDir + 'vdspd_20190728_20190729.csv'
        total_frames = cctvDir + "frame_num.txt"
        vd_df = pd.read_csv(filename, infer_datetime_format=True, index_col=0)
        vd_df.index = pd.to_datetime(vd_df.index)
        vd_df = vd_df.shift(-1)                              #往上位移一位 因為資料取得有延遲
        vd_df.loc[vd_df.index[len(vd_df)-1]] = 90            #將位移完後的nan填補90
        frame_df = pd.read_csv(total_frames, infer_datetime_format=True, index_col=0, header=None)
        frame_df.index = pd.to_datetime(frame_df.index)      #必須要用to_datetime 因為loc才能找到 從2019/07/29 ---> 2019-07-29
        total_test_index = []
        FrameList = []
        with open(cctvDir + "frame_interval.txt") as f:
            content = f.readlines()
        frame_interval = np.array([x.strip() for x in content], dtype=int)
        for i in range(len(content)):
            FrameList.append(FrameInterval())
        print(datetime.now())
        train_count = 0
        test_count = 0
        val_count = 0
        day = day1                       #first day of training day
        while day <= train_dayLast:
            # print(day)
            if not os.path.exists(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + ".npy")\
                    or day not in vd_df.index:
                day += timedelta(minutes=1)
                continue
            train_count += 1
            FrameList[int(frame_df.loc[day]) // 10] \
                .add_train(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + " 0", day)
            FrameList[int(frame_df.loc[day]) // 10] \
                .add_train(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + " 1", day)
             #     "D:/wcs/cctv/nfbCCTV-N5-S-21.048-M/20190729_1644 + " 0" or " 1" <- 有空白 , "2019-07-29 16:44:00"
            day += timedelta(minutes=1)                                      #"以方便後面做切割判斷"

        day = test_day1                 #first day of testing day
        while day <= dayLast:
            # print(day)
            if not os.path.exists(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + ".npy") \
                    or day not in vd_df.index:
                day += timedelta(minutes=1)
                continue
            test_count += 1
            FrameList[int(frame_df.loc[day]) // 10] \
                .add_test(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + " 0", day)
            total_test_index.append(day)
            day += timedelta(minutes=1)

        day = val_day1                  #first day of validating day
        while day <= val_dayLast:
            # print(day)
            if not os.path.exists(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + ".npy") \
                    or day not in vd_df.index:
                day += timedelta(minutes=1)
                continue
            val_count += 1
            FrameList[int(frame_df.loc[day]) // 10] \
                .add_val(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + " 0", day)
            day += timedelta(minutes=1)

        print("Total Train:", train_count, file=open(total_name, 'a'))
        print("Total test:", test_count, file=open(total_name, 'a'))
        print("Total val:", val_count, file=open(total_name, 'a'))
        print("Frame Num:", len(frame_df))

        ModelTrain_1 = []
        ModelTest_1 = []
        ModelTrain_2 = []
        ModelTest_2 = []
        with open('frame_index.txt', 'r') as f:        #frame_index.txt"是手動編集的
            for line in f:
                if cctv_Id in line:
                    t1, t2, t3, t4 = line.split(",")[1:]
                    for i in t1.split("/"):
                        ModelTrain_1.append(int(i))
                    for i in t2.split("/"):
                        ModelTest_1.append(int(i))
                    for i in t3.split("/"):
                        ModelTrain_2.append(int(i))
                    for i in t4.split("/"):
                        ModelTest_2.append(int(i))

        train = [ModelTrain_1, ModelTrain_2]
        test = [ModelTest_1, ModelTest_2]
        # train = [ModelTrain_2]
        # test = [ModelTest_2]

        total_test = []

        for peaks in range(2):
            print("\n*Peak %d" % (peaks+1), file=open(total_name, 'a'))
            print("Frame Num:", frame_num, file=open(total_name, 'a'))
            print("Train List:", train[peaks], file=open(total_name, 'a'))
            print("Test List:", test[peaks], file=open(total_name, 'a'))
            my_model, predict = compare_mae(train[peaks], test[peaks], frame_num, "peak_%d" % (peaks+1), model)
            my_model.save("plot/" + name + "/" + cctv_Id + "/peak_%d" % (peaks+1) + "/model.h5")
            total_test += test[peaks]
            index = 0
            for t in test[peaks]:
                temp = predict[index:index+len(FrameList[t].test_df)].tolist()
                while len(temp) < len(FrameList[t].test_df):
                    temp.append(temp[-1])
                index += len(FrameList[t].test_df)
                FrameList[t].test_df.insert(loc=0, column='predict', value=temp)
        predict_df = pd.DataFrame(columns=['value'])
        for i in total_test_index:
            for j in total_test:
                if i in FrameList[j].test_df.index:
                    predict_df.loc[i] = [FrameList[j].test_df.loc[i]['predict']]
        df = pd.DataFrame(index=total_test_index)
        predict_all = np.array(predict_df.loc[total_test_index, 'value'].values)
        vd_all = np.array(vd_df.loc[total_test_index, feature].values)
        df.insert(loc=0, column='target', value=vd_all)
        df.insert(loc=0, column='predict', value=predict_df)
        df = df.replace(r'\s+', np.nan, regex=True).fillna(method='ffill')
        # Fill first predicted value if needed
        df = df.replace(r'\s+', np.nan, regex=True).fillna(method='bfill')
        df.to_csv("plot/" + name + "/" + cctv_Id + "/total_result.csv")
        print("Total predict:", len(predict_df), file=open(total_name, 'a'))
        print("Total test:", len(vd_all), file=open(total_name, 'a'))
        y = np.array(df['target'].values)
        p = np.array(df['predict'].values)
        mae = abs(y - p).mean()
        plt.figure(num=None, figsize=(50, 5))
        std = abs(y - p).std()
        plt.title('MAE: %.2f, σ: %.2f' % (mae, std), fontsize=30)
        plt.tick_params(labelsize=30)
        plt.plot(y, label='Target')
        plt.plot(p, label='Predict')
        plt.legend(loc=3, prop={'size': 20})
        plt.tight_layout()
        plt.grid()
        # plt.title(mae_2)
        plt.savefig("plot/" + name + "/" + cctv_Id + "/total.png")
        plt.close('all')
        plt.clf()
        plt.cla()
        print("Total Std: %.2f" % std, file=open(total_name, 'a'))
        print("Total MAE: %.2f\n" % mae, file=open(total_name, 'a'))
        print("**********************\n", file=open(total_name, 'a'))
    model_count += 1
